In [ ]:
!pip install opencv-python-headless numpy PyMuPDF


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 20.0/20.0 MB 27.2 MB/s eta 0:00:00


In [ ]:
from google.colab import files
uploaded = files.upload()


Saving original.pdf to original.pdf


In [ ]:
import cv2
import numpy as np
import fitz  # PyMuPDF
import os

def pdf_to_images(pdf_path, output_dir="pages"):
    os.makedirs(output_dir, exist_ok=True)
    doc = fitz.open(pdf_path)
    image_paths = []
    for i, page in enumerate(doc):
        pix = page.get_pixmap(dpi=200)
        img_path = os.path.join(output_dir, f"page_{i+1}.png")
        pix.save(img_path)
        image_paths.append(img_path)
    return image_paths
def make_scanned_effect(img_path):
    img = cv2.imread(img_path)
    h, w = img.shape[:2]
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    noise = np.random.normal(0, 10, gray.shape).astype(np.uint8)
    noisy_img = cv2.add(gray, noise)
    M = cv2.getRotationMatrix2D((w // 2, h // 2), angle=0.7, scale=1)
    rotated = cv2.warpAffine(noisy_img, M, (w, h), borderMode=cv2.BORDER_CONSTANT, borderValue=255)
    blurred = cv2.GaussianBlur(rotated, (3, 3), 0)

    # Save in same directory, just add "_scanned" to filename
    base, ext = os.path.splitext(img_path)
    scanned_path = f"{base}_scanned{ext}"
    cv2.imwrite(scanned_path, blurred)
    return scanned_path

def images_to_pdf(image_paths, output_pdf="scanned_output.pdf"):
    doc = fitz.open()
    for path in image_paths:
        img = fitz.open(path)
        rect = img[0].rect
        pdfbytes = img.convert_to_pdf()
        img_pdf = fitz.open("pdf", pdfbytes)
        doc.insert_pdf(img_pdf)
    doc.save(output_pdf)
    doc.close()

def fake_scan_pdf(input_pdf, output_pdf="scanned_output.pdf"):
    print("Converting PDF to images...")
    pages = pdf_to_images(input_pdf)
    print("Applying scan-like effect...")
    scanned_pages = [make_scanned_effect(p) for p in pages]
    print("Converting images back to PDF...")
    images_to_pdf(scanned_pages, output_pdf)
    print(f"✅ Done! Fake scanned PDF saved as: {output_pdf}")


In [ ]:
fake_scan_pdf("original.pdf")


Converting PDF to images...
Applying scan-like effect...
Converting images back to PDF...
✅ Done! Fake scanned PDF saved as: scanned_output.pdf


In [ ]:
files.download("scanned_output.pdf")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>